In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("pima_indian.csv")
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
def normalize_feature_z_score(df):
    df1 = df.iloc[:,:-1].apply(lambda column:(column - column.mean())/column.std())
    return df1.join(df.iloc[:,-1])

def normalize_feature(df):
    df1 = df.iloc[:,:-1].apply(lambda column:((column - column.min()) /( column.max() - column.min()) - 0.5))
    return df1.join(df.iloc[:,-1])          

In [5]:
def hingeloss(W, X, y, C, grad = True):
    m, n = X.shape
    predict = np.dot(X, W)
    predict_correct = predict[np.arange(m),y].reshape(m, 1)
    hingescore = predict - predict_correct + 1.0
    hingescore[hingescore<0] = 0.0
    hingescore[np.arange(m), y] = 0.0
    loss = C * hingescore.sum() / m + (W ** 2).sum()
    if grad:
        hingescore[hingescore > 0] = 1.0
        hingescore[np.arange(m), y] = -np.sum(hingescore, axis=1)
        grad_W = C * np.dot(X.T, hingescore) / m + W
        return loss, grad_W
    return loss

def BGD(X, y, class_num=2, epoch=500, lr=0.1, C=1):
    m, n = X.shape
    loss_data = []
    W = np.random.randn(n,class_num)
    for i in range(epoch):
        loss, grad_W = hingeloss(W, X, y, C)
        loss_data.append(loss)
        W -= lr * grad_W
        if i%100==99:
            print('epoch %d: loss:%f' % (i, loss))
    return W, loss_data

def SGD(X, y, class_num, epoch=500, lr=0.1, C=1, batch_size=64):
    m, n = X.shape
    loss_data = []
    W = np.random.randn(n, class_num)
    indexs = np.arange(m)
    for i in range(epoch):
        np.random.shuffle(indexs)
        indices = np.array_split(indexs, m / batch_size)
        for index in indices:
            X_index = X[index]
            y_index = y[index]
            loss, grad_W = hingeloss(W, X_index, y_index, C)
            loss_data.append(loss)
            W -= lr * grad_W        
        if i%100 == 99:
                print('epoch %d: loss:%f' % (i, loss))
    return W, loss_data
        
def momentum(X, y, class_num, epoch=500, lr=0.1, C=1, beta=0.9):
    m, n = X.shape
    loss_data = []
    W = np.random.randn(n,class_num)
    v = np.zeros((n,class_num))
    for i in range(epoch):
        loss, grad_W = hingeloss(W, X, y, C)
        loss_data.append(loss)
        v = beta * v + (1-beta) * grad_W
        W -= lr * v
        if i%100==99:
            print('epoch %d: loss:%f' % (i, loss))
    return W, loss_data  

   
def RMSprop(X, y, class_num, epoch=500, lr=0.1, C=1, beta=0.999, eps=1e-8):
    m, n = X.shape
    loss_data = []
    W = np.random.randn(n,class_num)
    s = np.zeros((n,class_num))
    for i in range(epoch):
        loss, grad_W = hingeloss(W, X, y, C)
        loss_data.append(loss)
        s = beta * s + (1-beta) * grad_W ** 2
        W -= lr * grad_W / np.sqrt(s+eps)
        if i%100==99:
            print('epoch %d: loss:%f' % (i, loss))
    return W, loss_data 

def Adam(X, y, class_num, epoch=500, lr=0.1, C=1, beta_1=0.9, beta_2=0.999, eps=1e-8):
    m, n = X.shape
    loss_data = []
    W = np.random.randn(n,class_num)
    v = np.zeros((n,class_num))
    s = np.zeros((n,class_num))
    for i in range(epoch):
        loss, grad_W = hingeloss(W, X, y, C)
        loss_data.append(loss)
        v = beta_1 * v + (1-beta_1) * grad_W
        s = beta_2 * s + (1-beta_2) * grad_W ** 2
        v_corrected = v / (1 - beta_1 ** epoch)
        s_corrected = s / (1 - beta_2 ** epoch)
        W -= lr * v_corrected / np.sqrt(s_corrected+eps)
        if i%100==99:
            print('epoch %d: loss:%f' % (i, loss))
    return W, loss_data   

In [6]:
class SVC(object):
    def __init__(self, C, optimizer=BGD,class_num=2, fit_intercept=True):
        self.W = None
        self.C = C
        self.optimizer = optimizer
        self.fit_intercept = fit_intercept
        self.class_num = class_num

    def fit(self, X, y, epoch=500, lr=0.01):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        W, loss_data = self.optimizer(X, y,self.class_num, epoch, lr, self.C)
        self.W = W
        
        return loss_data

    def predict(self, X):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        value = np.dot(X, self.W) 
        predictions = np.argmax(value, axis=1)
        return predictions

In [7]:
df_nol = normalize_feature_z_score(data)
X = df_nol.iloc[:450,:-1]
y = df_nol.iloc[:450,-1]
X_val = df_nol.iloc[450:600,:-1]
y_val = df_nol.iloc[450:600,-1]
X_test = df_nol.iloc[601:,:-1]
y_test = df_nol.iloc[601:, -1]

In [10]:
acc = {}
for c in np.linspace(0.5, 2.0, 5):
    for opt in [BGD,SGD,momentum,RMSprop,Adam]:
        model = SVC(C=c, optimizer=opt)
        loss = model.fit(X, y,epoch=400)
        predictions = model.predict(X_val)
        acc[(c,opt)] = np.mean(predictions==y_val)

epoch 99: loss:2.017127
epoch 199: loss:0.637605
epoch 299: loss:0.479593
epoch 399: loss:0.460419
epoch 99: loss:0.476418
epoch 199: loss:0.448518
epoch 299: loss:0.503864
epoch 399: loss:0.470504
epoch 99: loss:2.098647
epoch 199: loss:0.578920
epoch 299: loss:0.467998
epoch 399: loss:0.458699
epoch 99: loss:0.452404
epoch 199: loss:0.453361
epoch 299: loss:0.454482
epoch 399: loss:0.453650
epoch 99: loss:0.557989
epoch 199: loss:0.456650
epoch 299: loss:0.456869
epoch 399: loss:0.456942
epoch 99: loss:3.505431
epoch 199: loss:0.972944
epoch 299: loss:0.751619
epoch 399: loss:0.729770
epoch 99: loss:0.724255
epoch 199: loss:0.724951
epoch 299: loss:0.700754
epoch 399: loss:0.686206
epoch 99: loss:3.039078
epoch 199: loss:0.912429
epoch 299: loss:0.743272
epoch 399: loss:0.729304
epoch 99: loss:0.723735
epoch 199: loss:0.722748
epoch 299: loss:0.722233
epoch 399: loss:0.723681
epoch 99: loss:0.739963
epoch 199: loss:0.728010
epoch 299: loss:0.727980
epoch 399: loss:0.727990
epoch 99: 

In [11]:
for c, accs in acc.items():
    print("c:%s, acc:%5f" % (c, accs))

c:(0.5, <function BGD at 0x7f6d6dfb4e18>), acc:0.780000
c:(0.5, <function SGD at 0x7f6d6df18048>), acc:0.780000
c:(0.5, <function momentum at 0x7f6d6df180d0>), acc:0.766667
c:(0.5, <function RMSprop at 0x7f6d6df18158>), acc:0.773333
c:(0.5, <function Adam at 0x7f6d6df181e0>), acc:0.780000
c:(0.875, <function BGD at 0x7f6d6dfb4e18>), acc:0.793333
c:(0.875, <function SGD at 0x7f6d6df18048>), acc:0.793333
c:(0.875, <function momentum at 0x7f6d6df180d0>), acc:0.793333
c:(0.875, <function RMSprop at 0x7f6d6df18158>), acc:0.793333
c:(0.875, <function Adam at 0x7f6d6df181e0>), acc:0.793333
c:(1.25, <function BGD at 0x7f6d6dfb4e18>), acc:0.793333
c:(1.25, <function SGD at 0x7f6d6df18048>), acc:0.800000
c:(1.25, <function momentum at 0x7f6d6df180d0>), acc:0.793333
c:(1.25, <function RMSprop at 0x7f6d6df18158>), acc:0.800000
c:(1.25, <function Adam at 0x7f6d6df181e0>), acc:0.793333
c:(1.625, <function BGD at 0x7f6d6dfb4e18>), acc:0.813333
c:(1.625, <function SGD at 0x7f6d6df18048>), acc:0.813333